In [188]:
import pandas as pd
from collections import defaultdict
import math
from typing import List, Dict
from bisect import bisect_left


In [189]:
# Global variables
currencies : List[str] = list()

In [190]:
df = pd.read_csv("./dataset/current_price.csv")
# df = pd.read_csv("./dataset/mock.csv")
df.shape

(136, 9)

In [191]:
df.head(3)

,Financial instrument,Current price,Change(%),Open,High,Low,Volume,Cap.,Issued Cap.
0,AUD/CAD,0.91530,-0.12%,0.91650,0.91710,0.91380,"86,310",-,-
1,AUD/CHF,0.57110,+0.56%,0.56790,0.57320,0.56670,"88,365",-,-
2,AUD/CNH,4.68318,-0.01%,4.68361,4.69713,4.67445,"319,202",-,-


In [192]:
# Pre-process data
column_names = list(df.columns)
pairs = df[column_names[0]].unique()
currency_set = set()
for pair in pairs:
    # create nodes
    numerator, denominator = str(pair).split("/")
    currency_set.add(numerator)
    currency_set.add(denominator)
currencies = list(currency_set)

In [193]:
# Swap USD (or any chosen base currency) to the first element
currencies.sort()
index = bisect_left(currencies, 'USD')
tmp = currencies[0]
currencies[0] = 'USD'
currencies[index] = tmp
currencies

['USD',
 'CAD',
 'CHF',
 'CNH',
 'CZK',
 'DKK',
 'EUR',
 'GBP',
 'HKD',
 'HUF',
 'ILS',
 'JPY',
 'MXN',
 'NOK',
 'NZD',
 'PLN',
 'SEK',
 'SGD',
 'TRY',
 'AUD',
 'ZAR']

In [194]:
# Mapping currency to index
index_mappings : Dict[str, int] = { key: i for i, key in enumerate(currencies) }

In [195]:
fx_rates = [[0.0 for j in range(len(currencies))]
            for i in range(len(currencies))]

for i in range(len(currencies)):
    fx_rates[i][i] = 1.0

# Analysis on the current price
for index, row in df.iterrows():
    pair = row[column_names[0]]
    numerator, denominator = str(pair).split("/")
    numIndex, denomIndex = index_mappings[numerator], index_mappings[denominator]
    fx_rates[numIndex][denomIndex] = row["Current price"]


for i in range(len(currencies) - 1):
    for j in range(i + 1, len(currencies)):
        if fx_rates[i][j] == 0 and fx_rates[j][i] != 0:
            fx_rates[i][j] = 1 / fx_rates[j][i] 
        elif fx_rates[i][j] != 0 and fx_rates[j][i] == 0:
            fx_rates[j][i] = 1 / fx_rates[i][j] 


# for r in fx_rates:
#     print(r)
# print(fx_rates)

Step 1: Convert FX rates to negative log weights

Suppose we have 3 currencies USD, EUR, and VND. To get the pair USD/VND, we multiply USD/EUR and EUR/VND. However, Bellman-Ford algorithm doesn't allow multiplication. Therefore, we utilize the logarithm property: `log(a) + log(b) = log(a*b)` to represent the edges.

With these edges, an arbitrage is a cycle with positive weight. Since Bellman-Ford algorithm is capable of finding negative-weight cycle, converting the edge weights to negative log should help us finding this cycle.

In [196]:
vertices = len(fx_rates)
edges = []

for i in range(vertices):
    for j in range(vertices):
        if i != j and fx_rates[i][j] != 0:
            weight = -math.log(fx_rates[i][j])
            edges.append((i, j, weight))

Helper function to trace the negative cycle path

In [197]:
def trace_negative_cycle(predecessor, start, vertices):
    cycle = []
    x = start

    # Move x back for a few steps in the cycle to ensure we reach the start of the cycle
    for _ in range(vertices):
        x = predecessor[x]

    # Trace the cycle
    cycle_start = x
    while True:
        cycle.append(x)
        x = predecessor[x]
        if x == cycle_start and len(cycle) > 1:
            cycle.append(x)  # Closing the cycle for clarity
            break

    cycle = list(reversed(cycle))
    cycle_str = [currencies[s] for s in cycle]
    # cycles.add('->'.join(str(x) for x in cycle))
    return '->'.join(cycle_str)

Step 2: Bellman-Ford function to detect negative cycles

In [198]:
def bellman_ford(vertices, edges, start):
    distance = [float("inf")] * vertices
    predecessor = [-1] * vertices
    distance[start] = 0

    # Relax edges (V-1) times
    for _ in range(vertices - 1):
        for u, v, weight in edges:
            if distance[u] != float("inf") and weight != 0 and distance[u] + weight < distance[v]:
                distance[v] = distance[u] + weight
                predecessor[v] = u

    # Check for negative cycle and trace it
    has_arbitrage = False
    for u, v, weight in edges:
        if distance[u] != float("inf") and weight != 0 and distance[u] + weight < distance[v]:
            # Trace the negative cycle detected
            has_arbitrage = True
            return has_arbitrage, trace_negative_cycle(predecessor, v, vertices)

    return has_arbitrage, ""

In [199]:
# Running the Bellman-Ford algorithm from start vertex (0/USD)
has_arbitrage, cycle = bellman_ford(vertices, edges, 0)
if has_arbitrage:
    print("Arbitrage opportunity detected in cycle.")
    amount = 100
    orders = cycle.split("->")
    for i in range(1, len(orders)):
        prev, cur = orders[i-1], orders[i]
        amount *= fx_rates[index_mappings[prev]][index_mappings[cur]]
    print("Cycle:", cycle)
    print("\tAmount:", amount)
    print("Profit: ", round((amount - 100) * 100, 2), "pips")
else:
    print("No arbitrage opportunity detected")

Arbitrage opportunity detected in cycle.
Cycle: PLN->JPY->MXN->AUD->PLN
	Amount: 100.02971275897569
Profit:  2.97 pips
